In [1]:
from pathlib import Path
from math import ceil
from textwrap import wrap
from io import BytesIO
from typing import Union, IO, Iterator, Callable, Tuple, List, Any
import re
import gzip
import logging
from nptyping import NDArray
import numpy as np
from matplotlib import pyplot as plt
from astropy.io import fits
from astropy.table import Table
from astropy.units import Quantity
import hpmoc as hpm
from hpmoc.healpy import healpy as hp
from hpmoc import PartialUniqSkymap
from hpmoc import utils as ut
from hpmoc.fits import (
    next_header,
    next_hdu,
    bintable_dtype,
    _next_header_blocks,
    read_bintable_chunks,
)

LOGGER = logging.getLogger(__name__)
DATA = Path(".").absolute().parent/"tests"/"data"
FITS_BLOCKSIZE = 2880
FITS_HEADER_WIDTH = 80
FITS_HEADER_KEY_LENGTH = 8
FITS_HEADER_LENGTH = FITS_BLOCKSIZE // FITS_HEADER_WIDTH
BINTABLE_TO_NUMPY_TYPES = {
    'L': '?',    # boolean
    # 'X': bit, not supported for now.
    'B': 'B',    # unsigned byte
    'I': '>i2',  # 16-bit big-endian int
    'J': '>i4',  # 32-bit big-endian int
    'K': '>i8',  # 64-bit big-endian int
    'A': 'b',    # char/signed byte
    'E': '>f4',  # 32-bit big-endian floating point
    'D': '>f8',  # 64-bit big-endian floating point
    'C': '>c8',  # 64-bit big-endian complex floating point
    'M': '>c16', # 128-bit big-endian complex floating point
    # 'P': 32-bit array descriptor, not supported for now
    # 'Q': 64-bit array descriptor, not supported for now
}
TFORM = re.compile(r'([0-9]*)([LXBIJKAEDCMPQ])(.*)')
BUFFER_ROWS = 4**8

NUNIQ_FITS = DATA/'S200105ae.fits'
NUNIQ_FITS_GZ = DATA/'S200105ae.fits.gz'
BAYESTAR_NEST_FITS_GZ_1024 = DATA/'S200316bj-1-Preliminary.fits.gz'
CWB_NEST_FITS_GZ_128 = DATA/'S200114f-3-Initial.fits.gz'
CWB_RING_FITS_GZ_128 = DATA/'S200129m-3-Initial.fits.gz'

In [56]:
def extract_probdensity(hdu, chunk, offset):
    "Meant to handle BAYESTAR MOC and NEST + CWB skymaps."
    ordering = hdu.header['ORDERING']
    if ordering == 'NUNIQ':
        u = chunk['UNIQ']
    else:
        for probname in ('PROBDENSITY', 'PROBABILITY', 'PROB'):
            if probname in hdu.columns.names:
                fmt = TFORM.match(hdu.columns[probname].format)[1]
                repeat = int(fmt) if fmt else 1
                break
        else:
            raise ValueError("Could not find a probability column in "
                             f"HDU columns: {hdu.columns}")
        nside = hdu.header['NSIDE']
        inds = np.arange(repeat*offset, repeat*(offset+len(chunk)))
        if ordering == 'RING':
            inds = hp.ring2nest(nside, inds)
        elif ordering != 'NESTED':
            raise ValueError(f"Unrecognized ordering: {ordering}")
        u = ut.nest2uniq(inds, nside, in_place=True)
    if 'PROBDENSITY' in hdu.columns.names:
        return u, [Quantity(chunk['PROBDENSITY'], copy=False,
                            unit=hdu.columns['PROBDENSITY'].unit)]
    LOGGER.debug(f"PROBDENSITY not found in {table}, trying to calculate "
                 f"it from PROB column and NSIDE")
    if 'PROBABILITY' in hdu.columns.names:
        LOGGER.debug("Column named PROBABILITY found. Using for probability "
                     "(Fermi GBM convention?)")
        probname = 'PROBABILITY'
    elif 'PROB' in hdu.columns.names:
        probname = 'PROB'
    else:
        raise ValueError(f"PROB not found in HDU columns: {hdu.columns}")
    probunit = hdu.columns[probname].unit
    if not probunit == 'pix-1':
        raise ValueError(f"Unexpected unit for {probname} column: {probunit}")
    prob = chunk[probname].ravel()  # handle CWB skymaps with repeated cols
    if ordering == 'NUNIQ':
        nside = ut.uniq2nside(u)
    return u, [Quantity(prob/ut.nside2pixarea(nside, degrees=False),
                        copy=False, unit='sr-1')]

In [57]:
def calculate_max_rows_read(hdu, dtype, buf_rows):
    "Meant to handle BAYESTAR MOC and NEST + CWB skymaps."
    for name, coltype, *repeat in dtype:
        if name in ('PROBDENSITY', 'PROB', 'PROBABILITY'):
            if repeat:
                return buf_rows // repeat[0]
            return buf_rows
    raise ValueError(f"Could not find a PROBABILITY column in {hdu.columns}, "
                     "do not know how to proceed.")

In [58]:
def read_bintable_chunks(
        stream: IO,
        tables: int = -1,
        buf_rows: int = BUFFER_ROWS,
        extractor: Callable[
            ['astropy.io.fits.BinTableHDU', NDArray[(Any,), Any], int],
            Tuple[
                NDArray[(Any,), int],
                List['astropy.units.Quantity'],
            ]
        ] = extract_probdensity,
        row_calculator: Callable[
            [
                'astropy.io.fits.BinTableHDU',
                List[Union[Tuple[str, str], Tuple[str, str, Tuple[int]]]],
                int,
            ],
            int
        ] = calculate_max_rows_read,
) -> Iterator[Iterator[Tuple[NDArray[(Any,), Any], List['astropy.units.Quantity']]]]:
    # assume first HDU has been skipped already
    while tables != 0:
        try:
            hdu = next_hdu(stream)
        except ut.EmptyStream:
            break
        dtype = bintable_dtype(hdu)
        max_rows = row_calculator(hdu, dtype, buf_rows)
        total_rows = hdu.header['NAXIS2']
        width = hdu.header['NAXIS1']
        assert total_rows * width == hdu.size
        
        # create a generator for each HDU
        def chunk_iter():
            offset = 0
            while offset < total_rows:
                rows = min(max_rows, total_rows - offset)
                chunk = np.frombuffer(stream.read(width*rows), dtype=dtype)
                yield extractor(hdu, chunk, offset)
                offset += rows
            assert offset == total_rows
            position_in_block = hdu.size % FITS_BLOCKSIZE
            if position_in_block != 0:
                stream.read(FITS_BLOCKSIZE - position_in_block)

        yield chunk_iter()
        tables -= 1

In [2]:
with gzip.open(CWB_NEST_FITS_GZ_128, 'rb') as f:
    # read first hdu
    hdu0 = next_hdu(f)
    for table in read_bintable_chunks(f):#, tables=1):
        for u, [s] in table:
            print(u)
            print(len(u))
            print(s)
            print(len(s))

[ 65536  65537  65538 ... 131069 131070 131071]
65536
[0. 0. 0. ... 0. 0. 0.] 1 / sr
65536
[131072 131073 131074 ... 196605 196606 196607]
65536
[0. 0. 0. ... 0. 0. 0.] 1 / sr
65536
[196608 196609 196610 ... 262141 262142 262143]
65536
[0. 0. 0. ... 0. 0. 0.] 1 / sr
65536


In [6]:
with open(NUNIQ_FITS, 'rb') as f:
    # read first hdu
    hdu0 = next_hdu(f)
    for table in read_bintable_chunks(f):#, tables=1):
        for u, [s] in table:
            print(u)
            print(len(u))
            print(s)
            print(len(s))

[  1024   1025   1026 ... 849929 849930 849931]
19200
[0.00924442 0.01020881 0.0026877  ... 2.03120541 2.07371061 1.95013386] 1 / sr
19200


In [5]:
with gzip.open(NUNIQ_FITS_GZ, 'rb') as f:
    # read first hdu
    hdu0 = next_hdu(f)
    for table in read_bintable_chunks(f):#, tables=1):
        for u, [s] in table:
            print(u)
            print(len(u))
            print(s)
            print(len(s))

[  1024   1025   1026 ... 849929 849930 849931]
19200
[0.00924442 0.01020881 0.0026877  ... 2.03120541 2.07371061 1.95013386] 1 / sr
19200


In [3]:
with gzip.open(CWB_RING_FITS_GZ_128, 'rb') as f:
    # read first hdu
    hdu0 = next_hdu(f)
    for table in read_bintable_chunks(f):#, tables=1):
        for u, [s] in table:
            print(u)
            print(len(u))
            print(s)
            print(len(s))

[ 81919  98303 114687 ... 178521 178518 178517]
65536
[1.6891969e-05 2.2008278e-05 1.8477627e-05 ... 4.5172563e-08 0.0000000e+00
 0.0000000e+00] 1 / sr
65536
[177834 177833 177830 ... 166233 166230 166229]
65536
[0. 0. 0. ... 0. 0. 0.] 1 / sr
65536
[165546 165545 165542 ... 212992 229376 245760]
65536
[0. 0. 0. ... 0. 0. 0.] 1 / sr
65536


In [4]:
with gzip.open(BAYESTAR_NEST_FITS_GZ_1024, 'rb') as f:
    # read first hdu
    hdu0 = next_hdu(f)
    for table in read_bintable_chunks(f):#, tables=1):
        for u, [s] in table:
            print(u)
            print(len(u))
            print(s)
            print(len(s))

[4194304 4194305 4194306 ... 4259837 4259838 4259839]
65536
[1.27585723e-04 1.27585723e-04 1.27585723e-04 ... 7.96194452e-01
 7.96194452e-01 7.96194452e-01] 1 / sr
65536
[4259840 4259841 4259842 ... 4325373 4325374 4325375]
65536
[1.58680076e-04 1.58680076e-04 1.58680076e-04 ... 4.87176649e-01
 4.87176649e-01 4.87176649e-01] 1 / sr
65536
[4325376 4325377 4325378 ... 4390909 4390910 4390911]
65536
[0.52377656 0.52377656 0.52377656 ... 0.00202571 0.00202571 0.00202571] 1 / sr
65536
[4390912 4390913 4390914 ... 4456445 4456446 4456447]
65536
[0.97938702 0.97938702 0.97938702 ... 0.00660068 0.00660068 0.00660068] 1 / sr
65536
[4456448 4456449 4456450 ... 4521981 4521982 4521983]
65536
[1.16985974e-04 1.16985974e-04 1.16985974e-04 ... 2.39931661e+00
 2.39931661e+00 2.39931661e+00] 1 / sr
65536
[4521984 4521985 4521986 ... 4587517 4587518 4587519]
65536
[2.97508683e-04 2.97508683e-04 2.97508683e-04 ... 5.47791355e-01
 5.47791355e-01 5.47791355e-01] 1 / sr
65536
[4587520 4587521 4587522 ... 4

[7995392 7995393 7995394 ... 8060925 8060926 8060927]
65536
[1.44072772e-07 1.44072772e-07 1.44072772e-07 ... 1.81438341e-08
 1.81438341e-08 1.81438341e-08] 1 / sr
65536
[8060928 8060929 8060930 ... 8126461 8126462 8126463]
65536
[4.17384785e-08 4.17384785e-08 4.17384785e-08 ... 2.03704334e-08
 2.03704334e-08 2.03704334e-08] 1 / sr
65536
[8126464 8126465 8126466 ... 8191997 8191998 8191999]
65536
[3.06762397e-09 3.06762397e-09 3.06762397e-09 ... 2.89459935e-08
 2.89459935e-08 2.89459935e-08] 1 / sr
65536
[8192000 8192001 8192002 ... 8257533 8257534 8257535]
65536
[3.61010899e-09 3.61010899e-09 3.61010899e-09 ... 6.01957965e-09
 6.01957965e-09 6.01957965e-09] 1 / sr
65536
[8257536 8257537 8257538 ... 8323069 8323070 8323071]
65536
[5.72206741e-08 5.72206741e-08 5.72206741e-08 ... 8.37059275e-09
 8.37059275e-09 8.37059275e-09] 1 / sr
65536
[8323072 8323073 8323074 ... 8388605 8388606 8388607]
65536
[2.98691377e-08 2.98691377e-08 2.98691377e-08 ... 1.07591501e-08
 1.07591501e-08 1.0759150

[11862016 11862017 11862018 ... 11927549 11927550 11927551]
65536
[3.0862258e-06 3.0862258e-06 3.0862258e-06 ... 2.1456500e-07 2.1456500e-07
 2.1456500e-07] 1 / sr
65536
[11927552 11927553 11927554 ... 11993085 11993086 11993087]
65536
[1.48721076e-06 1.48721076e-06 1.48721076e-06 ... 4.28300113e-06
 4.28300113e-06 4.28300113e-06] 1 / sr
65536
[11993088 11993089 11993090 ... 12058621 12058622 12058623]
65536
[3.54834030e-07 3.54834030e-07 3.54834030e-07 ... 6.33322766e-07
 6.33322766e-07 6.33322766e-07] 1 / sr
65536
[12058624 12058625 12058626 ... 12124157 12124158 12124159]
65536
[8.35627197e-08 8.35627197e-08 8.35627197e-08 ... 4.42288089e-07
 4.42288089e-07 4.42288089e-07] 1 / sr
65536
[12124160 12124161 12124162 ... 12189693 12189694 12189695]
65536
[3.30827246e-06 3.30827246e-06 3.30827246e-06 ... 4.62497614e-06
 4.62497614e-06 4.62497614e-06] 1 / sr
65536
[12189696 12189697 12189698 ... 12255229 12255230 12255231]
65536
[6.56380303e-08 6.56380303e-08 6.56380303e-08 ... 1.28847266

[16121856 16121857 16121858 ... 16187389 16187390 16187391]
65536
[1.65034406e-06 1.65034406e-06 1.65034406e-06 ... 1.02682046e-07
 1.02682046e-07 1.02682046e-07] 1 / sr
65536
[16187392 16187393 16187394 ... 16252925 16252926 16252927]
65536
[5.05729478e-07 5.05729478e-07 5.05729478e-07 ... 1.15163630e-08
 1.15163630e-08 1.15163630e-08] 1 / sr
65536
[16252928 16252929 16252930 ... 16318461 16318462 16318463]
65536
[1.79700610e-03 1.79700610e-03 1.79700610e-03 ... 2.87081411e-07
 2.87081411e-07 2.87081411e-07] 1 / sr
65536
[16318464 16318465 16318466 ... 16383997 16383998 16383999]
65536
[2.20747442e-05 2.20747442e-05 2.20747442e-05 ... 8.51318417e-07
 8.51318417e-07 8.51318417e-07] 1 / sr
65536
[16384000 16384001 16384002 ... 16449533 16449534 16449535]
65536
[1.64352676e-06 1.64352676e-06 1.64352676e-06 ... 1.82531002e-08
 1.82531002e-08 1.82531002e-08] 1 / sr
65536
[16449536 16449537 16449538 ... 16515069 16515070 16515071]
65536
[4.51547291e-07 4.51547291e-07 4.51547291e-07 ... 1.50

In [7]:
with gzip.open(CWB_RING_FITS_GZ_128, 'rb') as f:
    hd0 = next_hdu(f)
    hd1 = next_hdu(f)

In [8]:
hd1.header

XTENSION= 'BINTABLE'           / binary table extension                         
BITPIX  =                    8 / 8-bit bytes                                    
NAXIS   =                    2 / 2-dimensional binary table                     
NAXIS1  =                 4096 / width of table in bytes                        
NAXIS2  =                  192 / number of rows in table                        
PCOUNT  =                    0 / size of special data area                      
GCOUNT  =                    1 / one data group (required keyword)              
TFIELDS =                    1 / number of fields in each row                   
TTYPE1  = 'PROB    '           / label for field   1                            
TFORM1  = '1024E   '           / data format of field: 4-byte REAL              
TUNIT1  = 'pix-1   '           / physical unit of field                         
EXTNAME = 'xtension'           / name of this binary table extension            
PIXTYPE = 'HEALPIX '        

In [11]:
hd1.size

192.0

In [41]:
with open(NUNIQ_FITS, 'rb') as f:
    hs0 = _next_header_blocks(f)
    hs1 = _next_header_blocks(f)
    hd0, = fits.HDUList.fromstring(hs0)
    hd1, = fits.HDUList.fromstring(hs1)
    dtype = np.dtype([(c.name, BINTABLE_TO_NUMPY_TYPES[c.format]) for c in hd1.columns])
    #tab = np.frombuffer(f.read(hd1.header['NAXIS1']*10), dtype=dtype)
    tab = np.frombuffer(f.read(hd1.header['NAXIS1']*10), dtype=[('UNIQ', '>i8'), ('REST', '(4,)>f8')])
    #h1 = fits.Header.fromstring(hs1)
    #h1['NAXIS2'] = 100
    #hd1_0, = fits.HDUList.fromstring(h1.tostring().encode())
    #buf = BytesIO(f.read(hd1_0.size))
    #buf.seek(0)
    #tab = hd1_0.readfrom(buf)

In [42]:
hd1.header

XTENSION= 'BINTABLE'           / binary table extension                         
BITPIX  =                    8 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                   40 / length of dimension 1                          
NAXIS2  =                19200 / length of dimension 2                          
PCOUNT  =                    0 / number of group parameters                     
GCOUNT  =                    1 / number of groups                               
TFIELDS =                    5 / number of table fields                         
TTYPE1  = 'UNIQ    '                                                            
TFORM1  = 'K       '                                                            
TTYPE2  = 'PROBDENSITY'                                                         
TFORM2  = 'D       '                                                            
TUNIT2  = 'sr-1    '        

In [102]:
tab

array([(1024, 0.00924442, 166.04100755,  77.64237103, 2.97840680e-05),
       (1025, 0.01020881, 143.73043837,  83.13282123, 3.63855617e-05),
       (1026, 0.0026877 , 132.46073881,  84.66111686, 4.06920609e-05),
       (1027, 0.00301602,  69.71120026,  99.29488834, 7.50446909e-05),
       (1028, 0.01326684, 134.5433377 ,  82.92336069, 4.02186414e-05),
       (1029, 0.0205938 , 144.78794411,  76.27714446, 3.74018580e-05),
       (1030, 0.00434119,  35.15745476, 103.1343964 , 1.12547108e-04),
       (1031, 0.0067673 ,  53.30797332,  94.44251615, 9.89361732e-05),
       (1032, 0.00083561,  97.43396584,  91.10788204, 5.79422459e-05),
       (1033, 0.00090659, -65.27190554, 123.42406103, 3.30732314e-04)],
      dtype=[('UNIQ', '>i8'), ('PROBDENSITY', '>f8'), ('DISTMU', '>f8'), ('DISTSIGMA', '>f8'), ('DISTNORM', '>f8')])

In [71]:
BytesIO(f.read

In [67]:
hd1_0.readfrom?

In [50]:
hd1.readfrom?

In [27]:
isinstance(hd1, fits.BinTableHDU)

True

In [33]:
h1 = fits.Header.fromstring(hs1)

In [36]:
h1['NAXIS2'] = 100

In [88]:
[(c.name, BINTABLE_TO_NUMPY_TYPES[c.format]) for c in hd1.columns]

dtype([('UNIQ', '>i8'), ('PROBDENSITY', '>f8'), ('DISTMU', '>f8'), ('DISTSIGMA', '>f8'), ('DISTNORM', '>f8')])

In [80]:
np.dtype([('x', '>i2')])

dtype([('x', '>i2')])

In [73]:
fits.Header.fromstring(hs1)

XTENSION= 'BINTABLE'           / binary table extension                         
BITPIX  =                    8 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                   40 / length of dimension 1                          
NAXIS2  =                19200 / length of dimension 2                          
PCOUNT  =                    0 / number of group parameters                     
GCOUNT  =                    1 / number of groups                               
TFIELDS =                    5 / number of table fields                         
TTYPE1  = 'UNIQ    '                                                            
TFORM1  = 'K       '                                                            
TTYPE2  = 'PROBDENSITY'                                                         
TFORM2  = 'D       '                                                            
TUNIT2  = 'sr-1    '        

In [5]:
pwd

'/Users/s/dev/hpmoc/jup'

In [ ]:
with open(NUNIQ_FITS, 'rb') as f:
    f.read(FITS_BLOCKSIZE)
    

In [ ]:
def load(infile: Union[IO, str], 